# Model Training

### Imports

In [19]:
#!pip install sagemaker boto3 smdebug sagemaker-experiments
#!pip install --upgrade awscli boto3

In [20]:
# import sagemaker
# import boto3
# import pandas as pd
# from sagemaker import get_execution_role, KMeans, Session
# from sagemaker import PCA
# from sklearn.preprocessing import StandardScaler
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.decomposition import IncrementalPCA
# from sklearn.preprocessing import StandardScaler
# from sagemaker import KMeans
# from sklearn.metrics import silhouette_score
# import time
# import json
# from smexperiments.experiment import Experiment
# from smexperiments.trial import Trial
# from smexperiments.trial_component import TrialComponent
# from smexperiments.tracker import Tracker
# from sagemaker.model import Model
# from sagemaker.serializers import CSVSerializer
# from sagemaker.deserializers import JSONDeserializer
# import os
# import tarfile

In [21]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import IncrementalPCA
# from sagemaker import PCA, KMeans, get_execution_role, Session
# from sklearn.metrics import silhouette_score
# import time

## Get Training Data From S3

In [22]:
# # Load the train dataset from S3
# train_df = pd.read_csv('s3://team6datasets/splitDatasets/train.csv')
# print(train_df.head())

## Train KNN Model

### Feature Selection with SKLearn IPCA & SageMaker Built-In PCA

In [23]:
# # Scaler
# scaler = StandardScaler()

# # Convert to float32 for memory efficiency
# scaled_data = scaler.fit_transform(train_df.astype('float32'))

# # Use IncrementalPCA
# ipca = IncrementalPCA(n_components=100, batch_size=150)  # Adjust batch_size
# ipca.fit(scaled_data)

# # Plot the explained variance ratio
# plt.figure(figsize=(10, 7))
# plt.bar(range(1, ipca.n_components_ + 1), ipca.explained_variance_ratio_, alpha=0.5, align='center',
#         label='Individual explained variance')
# plt.step(range(1, ipca.n_components_ + 1), np.cumsum(ipca.explained_variance_ratio_), where='mid',
#          label='Cumulative explained variance')
# plt.ylabel('Explained variance ratio')
# plt.xlabel('Principal component index')
# plt.legend(loc='best')
# plt.show()

In [24]:
# # Define PCA model
# pca_estimator = PCA(role=get_execution_role(),
#                     instance_count=1,
#                     instance_type='ml.c4.xlarge',
#                     num_components=10,
#                     sagemaker_session=sagemaker_session)

# # Fit PCA model
# pca_estimator.fit(pca_estimator.record_set(scaled_data))

In [25]:
# # access PCA artifacts
# model_artifacts_s3_uri = pca_estimator.model_data
# print("Model artifacts saved at:", model_artifacts_s3_uri)

In [26]:
# # Create a S3 client
# s3 = boto3.client('s3')
# bucket_name = model_artifacts_s3_uri.split('/')[2]
# object_key = '/'.join(model_artifacts_s3_uri.split('/')[3:])

# # Download the model artifacts file
# s3.download_file(bucket_name, object_key, 'model.tar.gz')

# # Extract the model artifacts file
# with tarfile.open('model.tar.gz') as tar:
#     tar.extractall()
#     print("Extracted model artifacts in the current directory")

In [27]:
# import mxnet as mx

# data = mx.ndarray.load('model_algo-1')
# # MXNet NDArrays named 's' usually contain the singular values, which can be related to explained variance

In [28]:
# # total variance ratios
# explained_variance_ratio = np.square(data['s'].asnumpy()) / np.sum(np.square(data['s'].asnumpy()))

# # Plotting the cumulative explained variance
# plt.figure(figsize=(10, 7))
# plt.plot(np.cumsum(explained_variance_ratio), marker='o', linestyle='-', color='b')
# plt.title('Cumulative Explained Variance by Principal Component')
# plt.xlabel('Number of Principal Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.grid(True)
# plt.show()

### Set Up Experiment for KNN Model Training Trials

In [29]:
# # set up experiment
# experiment_name = f"KMeans-Clustering-Experiment-{int(time.time())}"
# experiment_description = "Comparing 3 KMeans models with different hyperparameters."

# experiment = Experiment.create(
#     experiment_name=experiment_name,
#     description=experiment_description,
#     sagemaker_boto_client=boto3.client('sagemaker')
# )

In [30]:
# # define hyperparameters for 3 KNN models
# hyperparameter_sets = [
#     {'k': '5', 'init_method': 'random'},
#     {'k': '10', 'init_method': 'random'},
#     {'k': '15', 'init_method': 'random'}
# ]

### Run Experiment Trials for KNN Models

In [31]:
# # create my own S3 bucket
# s3 = boto3.client('s3')
# s3.create_bucket(Bucket='team6experiments')

In [32]:
# # Run trials
# role = get_execution_role()
# sagemaker_session = sagemaker.Session()
# region = sagemaker_session.boto_region_name

# for hyperparameters in hyperparameter_sets:
#     # Create a trial
#     trial_name = f"kmeans-trial-k-{hyperparameters['k']}-{int(time.time())}"
#     trial = Trial.create(
#         trial_name=trial_name, 
#         experiment_name=experiment.experiment_name,
#         sagemaker_boto_client=boto3.client('sagemaker'),
#     )
    
#     # define output path
#     output_path = f's3://team6experiments/{trial_name}/'
    
#     # Create the KMeans estimator
#     kmeans = KMeans(role=role,
#                     instance_count=1,
#                     instance_type='ml.c4.xlarge',
#                     k=int(hyperparameters['k']),
#                     sagemaker_session=sagemaker_session,
#                     output_path= output_path)

#     # Fit the model
#     kmeans.fit(kmeans.record_set(scaled_data), wait=True)

### Calculate Silhouette Scores for KNN Models

In [33]:
# # define model artifacts and hyperparameters
# model_artifacts = [
#     {
#         "model_data": "s3://team6experiments/kmeans-trial-k-5-1711914356/kmeans-2024-03-31-19-46-11-504/output/model.tar.gz", # need to get links to S3 artifacts first!
#         "hyperparameters": {"k": 5}
#     },
#     {
#         "model_data": "s3://team6experiments/kmeans-trial-k-10-1711914593/kmeans-2024-03-31-19-50-08-046/output/model.tar.gz", # need to get links to S3 artifacts first!
#         "hyperparameters": {"k": 10}
#     },
#     {
#         "model_data": "s3://team6experiments/kmeans-trial-k-15-1711914830/kmeans-2024-03-31-19-54-05-839/output/model.tar.gz", # need to get links to S3 artifacts first!
#         "hyperparameters": {"k": 15}
#     }
# ]

In [34]:
# # automate deployment, prediction, and evaluation for KNN models
# def deploy_predict_evaluate(model_data, hyperparameters):
#     print(f"Deploying model with data: {model_data}")
    
#     # Define model
#     model = Model(
#         model_data=model_data,
#         image_uri=sagemaker.image_uris.retrieve('kmeans', region),
#         role=role,
#         sagemaker_session=sagemaker_session
#     )
    
#     # Attempt to deploy the model
#     try:
#         predictor = model.deploy(initial_instance_count=1, instance_type="ml.t2.medium")
#         print("Model deployed successfully.")
#     except Exception as e:
#         print(f"An error occurred during model deployment: {e}")
#         return None
    
#     if predictor:
#         # Set serializer and deserializer
#         predictor.serializer = CSVSerializer()
#         predictor.deserializer = JSONDeserializer()
        
#         try:
#             # Make predictions
#             results = predictor.predict(scaled_data.tolist())
#             cluster_labels = np.array([r['closest_cluster'] for r in results['predictions']])
            
#             # Evaluate with Silhouette Score
#             score = silhouette_score(scaled_data, cluster_labels)
#             print(f"Silhouette Score for k={hyperparameters['k']}: {score}")
#         except Exception as e:
#             print(f"An error occurred during prediction or evaluation: {e}")
#             score = None
#         finally:
#             # Clean up
#             try:
#                 sagemaker.Session().delete_endpoint(predictor.endpoint_name)
#                 print("Endpoint deleted successfully.")
#             except Exception as e:
#                 print(f"Failed to delete endpoint {predictor.endpoint_name}: {e}")
#     else:
#         print("Deployment failed. Predictor is None.")
#         score = None
    
#     return score

# # Assuming 'model_artifacts' and 'scaled_data' are defined earlier in your code
# silhouette_scores = [deploy_predict_evaluate(artifact['model_data'], artifact['hyperparameters']) for artifact in model_artifacts]

# print("Silhouette scores:", silhouette_scores)

In [35]:
# # release end-point
# sagemaker.Session().delete_endpoint(predictor.endpoint_name)